# Replication Acemoglu, Akcegit y Kerr (2015)

## 0. Libraries

First, we import the libraries we need

In [1]:
import pandas as pd
import numpy as np
import random as rd
import math as mt
from linearmodels import PanelOLS


## 1. Introduction and some functions

We need to estimate the $\beta$-coefficients in the following equation:  
 
  $\Delta\ln Y_{i,t}=\delta_t+\psi\Delta\ln Y_{i,t-1}+\beta^{own}Shock_{i,t-1}+\beta^{upstream}Upstream_{i,t-1}+\beta^{downstream}Downstream_{i,t-1}+\varepsilon_{i,t}\;\;\forall_i$
 

### China Import Shocks

In this section, the shocks considered is the trade with china, i.e., Shock="ChinaTrade" and it is defined as: 
 
 $ChinaTrade^{IV}_{j,t}=-\dfrac{NonUS\;Imports\;From\;China_{j,t}}{US\;Market\;Size_{j,1991}}$

And, then, $Downstream_{i,t}^{TradeIV}=\sum_j(Input\%^{1991}_{j\rightarrow i}-1_{j=i})·\nabla ChinaTrade_{j,t}^{IV}$.
 
 By the other hand, $Upstream_{i,t}=\sum_j(Output\%_{i\rightarrow j}^{1991}-1_{j=i})·Shock_{j,t}$

## 2. Data

As we found the data in Acemoglu's webpage, we will import it here (it is in stata format). This is an example function to extract it.

**Descargamos todos los datos en el notebook**

In [195]:
data1 = pd.io.stata.read_stata('leontief_up.dta') #used
data2 = pd.io.stata.read_stata('aak-grid-sort-ext-sum.dta') #not nec
data3 = pd.io.stata.read_stata('AF_trade_variables.dta') #nec, for trade ups and downs
data4 = pd.io.stata.read_stata('all_sales.dta') #not used
data5 = pd.io.stata.read_stata('cbp_czone_1991.dta') #not used
data6 = pd.io.stata.read_stata('cite-for-trend2b.dta') #DK
data7 = pd.io.stata.read_stata('cite-us-trend2b.dta') #DK
data8 = pd.io.stata.read_stata('gross_shares.dta') # not used
data9 = pd.io.stata.read_stata('leontief_down.dta') ## used 
data10= pd.io.stata.read_stata('main_annual_format.dta') # ??? debe de estar aquí lo que necesito pero no me doy cuenta de ello
data11= pd.io.stata.read_stata('nber-sic5809.dta') # vadd and piship to rvadd
data12= pd.io.stata.read_stata('pce.dta') #prices, not used
data13= pd.io.stata.read_stata('sic87dd_trade_data.dta') #not used

#rvadd for yt


In [196]:
data3.columns

Index(['sic87dd', 'year', 'real_imports_usch', 'real_exports_usch',
       'real_netimports_us', 'l_import_usch', 'l_import_otch', 'l_export_usch',
       'l_export_otch', 'd_import_usch_1991_2011', 'd_import_otch_1991_2011',
       'd_export_usch_1991_2011', 'd_export_otch_1991_2011',
       'd_net_import_usch_1991_2011', 'd_net_import_otch_1991_2011',
       'd_import_usch_1991_1999', 'd_import_otch_1991_1999',
       'd_export_usch_1991_1999', 'd_export_otch_1991_1999',
       'd_net_import_usch_1991_1999', 'd_net_import_otch_1991_1999',
       'd_import_usch_1999_2007', 'd_import_otch_1999_2007',
       'd_export_usch_1999_2007', 'd_export_otch_1999_2007',
       'd_net_import_usch_1999_2007', 'd_net_import_otch_1999_2007',
       'd_import_usch_1999_2011', 'd_import_otch_1999_2011',
       'd_export_usch_1999_2011', 'd_export_otch_1999_2011',
       'd_net_import_usch_1999_2011', 'd_net_import_otch_1999_2011',
       'd_import_usch_2007_2011', 'd_import_otch_2007_2011',
       'd_

In [197]:
data3

,sic87dd,year,real_imports_usch,real_exports_usch,real_netimports_us,l_import_usch,l_import_otch,l_export_usch,l_export_otch,d_import_usch_1991_2011,...,d_export_usch_1991_2009,d_export_otch_1991_2009,d_net_import_usch_1991_2009,d_net_import_otch_1991_2009,d_import_usch_1999_2009,d_import_otch_1999_2009,d_export_usch_1999_2009,d_export_otch_1999_2009,d_net_import_usch_1999_2009,d_net_import_otch_1999_2009
0,2011.0,1991,2.013399e+07,NaN,-5.910849e+08,0.021218,0.195554,NaN,NaN,0.163523,...,0.799891,0.796011,-0.644384,-0.369059,0.126179,0.295903,0.483464,0.679257,-0.357285,-0.383353
1,2011.0,1992,2.360702e+07,2.913660e+07,-1.375761e+09,0.024878,0.253203,0.030706,0.085144,0.163523,...,0.799891,0.796011,-0.644384,-0.369059,0.126179,0.295903,0.483464,0.679257,-0.357285,-0.383353
2,2011.0,1993,2.041534e+07,3.270506e+07,-1.940466e+09,0.021515,0.230878,0.034466,0.093803,0.163523,...,0.799891,0.796011,-0.644384,-0.369059,0.126179,0.295903,0.483464,0.679257,-0.357285,-0.383353
3,2011.0,1994,2.335108e+07,9.941616e+07,-2.765599e+09,0.024608,0.241313,0.104769,0.119880,0.163523,...,0.799891,0.796011,-0.644384,-0.369059,0.126179,0.295903,0.483464,0.679257,-0.357285,-0.383353
4,2011.0,1995,2.330957e+07,1.686933e+08,-3.745626e+09,0.024565,0.308338,0.177777,0.167666,0.163523,...,0.799891,0.796011,-0.644384,-0.369059,0.126179,0.295903,0.483464,0.679257,-0.357285,-0.383353
5,2011.0,1996,2.832243e+07,1.925002e+08,-4.264464e+09,0.029848,0.328880,0.202866,0.177800,0.163523,...,0.799891,0.796011,-0.644384,-0.369059,0.126179,0.295903,0.483464,0.679257,-0.357285,-0.383353
6,2011.0,1997,3.746760e+07,2.138567e+08,-3.721887e+09,0.039485,0.339754,0.225372,0.198831,0.163523,...,0.799891,0.796011,-0.644384,-0.369059,0.126179,0.295903,0.483464,0.679257,-0.357285,-0.383353
7,2011.0,1998,4.948151e+07,2.540542e+08,-2.227170e+09,0.052146,0.352854,0.267734,0.168102,0.163523,...,0.799891,0.796011,-0.644384,-0.369059,0.126179,0.295903,0.483464,0.679257,-0.357285,-0.383353
8,2011.0,1999,4.796380e+07,2.872275e+08,-2.358846e+09,0.050547,0.336911,0.302694,0.163381,0.163523,...,0.799891,0.796011,-0.644384,-0.369059,0.126179,0.295903,0.483464,0.679257,-0.357285,-0.383353
9,2011.0,2000,5.201332e+07,4.264282e+08,-3.377562e+09,0.054814,0.358476,0.449390,0.288381,0.163523,...,0.799891,0.796011,-0.644384,-0.369059,0.126179,0.295903,0.483464,0.679257,-0.357285,-0.383353


In [198]:
data3=data3.loc[:,["sic87dd","year","l_import_otch"]] ## voy a probar con esto en diferencias

In [199]:
rvadd=pd.DataFrame()
rvadd["sic"]=data11["sic"]
rvadd["year"]=data11["year"]
rvadd["vadd"]=data11["vadd"]
rvadd["piship"]=data11["piship"]
rvadd["rvadd"]=rvadd["vadd"]/rvadd["piship"]
rvadd.head()

,sic,year,vadd,piship,rvadd
0,2011,1958.0,1748.6,0.403,4338.957847
1,2011,1959.0,1833.2,0.372,4927.956872
2,2011,1960.0,1910.7,0.360,5307.499789
3,2011,1961.0,1889.2,0.357,5291.876850
4,2011,1962.0,1986.1,0.368,5397.010859


We need real value added in changes!! (the equation is specified in changes)

In [200]:
rvadd=rvadd[rvadd["year"]>=1991]
rvadd=rvadd[rvadd["year"]<=2009]

In [201]:
rvadd_tensor=[]

In [202]:
i=0
while i<len(rvadd):
    rvadd_tensor.append(rvadd.iloc[i:i+19])
    i=i+19

In [203]:
len(rvadd_tensor)

459

In [204]:
#trying to fix the BIG ISSUE

**This (A_out) is $Output\%_{i\rightarrow j}$:**

In [205]:
A_out=pd.DataFrame()
indice=data1.groupby("sic87dd_up").count().index
for i in indice:
    A_out[i]=np.zeros(len(indice))
A_out["indice"]=indice
A_out=A_out.set_index("indice")

In [206]:
i=0
while i<len(data1):
    col   = data1["sic87dd_up"][i]
    row   = data1["sic87dd_down"][i]
    valor = data1["purchase_share"][i]
    A_out[row][col]=valor
    i=i+1

In [207]:
A_out.head()

,710,740,780,800,900,1010,1020,1030,1221,1410,...,8330,8350,8360,8610,8630,8660,8711,8720,8731,8741
indice,,,,,,,,,,,,,,,,,,,,,
710,0.010928,9.415244e-05,4.576397e-05,0.233058,6.826502e-05,6.215063e-05,1.679412e-04,2.474521e-04,4.183772e-05,1.080548e-04,...,1.743170e-04,1.051794e-04,5.375519e-05,1.999586e-04,1.614759e-04,6.019359e-05,3.057869e-05,4.691407e-05,4.931733e-05,3.940141e-05
740,0.000130,1.737029e-07,5.243625e-08,0.000030,1.906675e-07,3.511963e-07,3.036998e-07,7.817558e-07,8.433020e-08,1.997410e-07,...,3.873131e-07,2.209120e-07,6.140364e-07,5.345115e-07,6.439342e-07,1.023227e-07,1.251504e-07,5.862727e-07,2.632679e-07,1.362326e-07
780,0.000402,1.126960e-03,2.302349e-04,0.000657,1.908602e-03,1.585438e-03,1.113297e-03,3.151666e-03,3.537685e-04,8.911513e-04,...,1.732513e-03,2.123065e-03,2.970997e-03,2.576769e-03,3.260528e-03,2.979802e-03,6.509421e-04,2.187797e-03,1.060035e-03,7.195062e-04
800,0.002361,4.137816e-04,2.011237e-04,0.024245,3.000115e-04,2.731399e-04,7.380688e-04,1.087504e-03,1.838686e-04,4.748799e-04,...,7.660894e-04,4.622428e-04,2.362435e-04,8.787788e-04,7.096550e-04,2.645391e-04,1.343873e-04,2.061782e-04,2.167401e-04,1.731615e-04
900,0.000136,2.388501e-04,6.888517e-05,0.000173,2.367207e-01,1.230077e-04,1.602000e-04,2.018669e-04,9.365488e-05,1.494808e-04,...,1.496836e-04,1.495268e-04,2.848462e-03,1.843394e-04,3.171373e-04,6.793377e-05,9.317891e-05,1.442506e-04,1.871773e-04,4.029215e-04


In [208]:
identidad=pd.DataFrame()
for i in A_out.index:
    identidad[i]=np.zeros(len(A_out.index))
identidad["indice"]=A_out.index
identidad=identidad.set_index("indice")
i=0
while i<len(identidad):
    identidad.iloc[i,i]=1
    i=i+1
hinter=identidad-A_out
h_out = pd.DataFrame(np.linalg.pinv(hinter.values), hinter.columns, hinter.index)
#por algún motivo son todos negativos?????? (les cambio el signo, but...)
A_out=h_out #renombro para no tener que cambiar nada más

In [209]:
A_out.head()

indice,710,740,780,800,900,1010,1020,1030,1221,1410,...,8330,8350,8360,8610,8630,8660,8711,8720,8731,8741
710,1.016235,0.005521,0.001684,0.245215,0.004615,0.004027,0.006318,0.011749,0.002195,0.003998,...,0.005137,0.004557,0.003942,0.009092,0.007877,0.002943,0.002378,0.003314,0.003011,0.002927
740,0.000401,1.000006,0.000002,0.000191,0.000005,0.000005,0.000007,0.000015,0.000003,0.000005,...,0.000006,0.000005,0.000006,0.000010,0.000009,0.000003,0.000003,0.000005,0.000004,0.000004
780,0.003692,0.004971,1.001496,0.003446,0.005043,0.004531,0.005330,0.011611,0.001983,0.003704,...,0.004786,0.005161,0.006205,0.007216,0.008242,0.005663,0.002817,0.005026,0.003384,0.003174
800,0.012473,0.012006,0.003663,1.033238,0.010036,0.008758,0.013738,0.025550,0.004772,0.008693,...,0.011172,0.009910,0.008573,0.019771,0.017130,0.006399,0.005170,0.007207,0.006548,0.006364
900,0.001738,0.002299,0.000664,0.001827,1.311545,0.001533,0.002076,0.003545,0.000937,0.001590,...,0.001375,0.001377,0.006423,0.002074,0.002352,0.000776,0.000940,0.001260,0.001327,0.001927


In [210]:
#trying to fix the BIG ISSUE

**This (A_in) is $Input\%_{j\rightarrow i}$:**

In [211]:
A_in=pd.DataFrame()
indice=data9.groupby("sic87dd_up").count().index
for i in indice:
    A_in[i]=np.zeros(len(indice))
A_in["indice"]=indice
A_in=A_in.set_index("indice")

In [212]:
i=0
while i<len(data9):
    row   = data9["sic87dd_up"][i]
    col   = data9["sic87dd_down"][i]
    valor = data9["produce_share"][i]
    A_in[row][col]=valor
    i=i+1

In [213]:
identidad=pd.DataFrame()
for i in A_in.index:
    identidad[i]=np.zeros(len(A_in.index))
identidad["indice"]=A_in.index
identidad=identidad.set_index("indice")
i=0
while i<len(identidad):
    identidad.iloc[i,i]=1
    i=i+1
hinter=identidad-A_in
h_in = pd.DataFrame(np.linalg.pinv(hinter.values), hinter.columns, hinter.index)
#por algún motivo son todos negativos?????? (les cambio el signo, but...)
A_in=h_in #renombro para no tener que cambiar nada más

In [214]:
A_in.head()

indice,710,740,780,800,900,1010,1020,1030,1221,1410,...,8330,8350,8360,8610,8630,8660,8711,8720,8731,8741
710,1.016235,0.000801,0.003100,0.020739,0.001505,0.019675,0.026421,0.009595,0.017133,0.005912,...,0.000913,6.787106e-17,6.223320e-17,0.005808,0.000031,-2.764716e-17,0.004835,0.007790,0.003364,0.005544
740,0.002762,1.000006,0.002088,0.009988,0.000996,0.008019,0.012930,0.004656,0.006383,0.002290,...,0.000802,-1.011019e-16,-2.919214e-16,0.005337,0.000027,-1.219998e-16,0.002523,0.005393,0.002422,0.003517
780,0.002006,0.000005,1.001496,0.007254,0.000684,0.007087,0.010576,0.003816,0.006923,0.002182,...,0.000421,-3.328501e-16,1.301043e-17,0.003166,0.000018,2.602085e-18,0.002092,0.003789,0.002138,0.003140
800,0.147481,0.000230,0.001740,1.033238,0.000951,0.008995,0.012471,0.004550,0.007381,0.002542,...,0.000452,8.131516e-18,-3.218725e-17,0.003019,0.000015,2.358140e-17,0.002202,0.007921,0.001664,0.002854
900,0.005332,0.000012,0.004892,0.019279,1.311545,0.015620,0.024188,0.008904,0.015924,0.005144,...,0.000919,4.336809e-17,-2.580401e-17,0.005688,0.000032,-2.526191e-17,0.004761,0.006669,0.003269,0.005271


Necesitamos quedarnos sólo con las industrias coincidentes en índice de A_in y A_out y nber:

In [215]:
indicervadd=rvadd.groupby("sic").count().index
indicea=A_in.index
#buscamos su intersección
indiceinter=np.intersect1d(indicea,indicervadd)
print("There are "+str(len(indiceinter))+" industries")

There are 392 industries


In [216]:
Input=A_in.loc[indiceinter,indiceinter]
Output=A_out.loc[indiceinter,indiceinter]

In [217]:
def esta_en(num,array):
    esta=False
    result=False
    while not esta:
        for j in array:
            if num==j:
                esta=True
                result=True
        esta=True
    return result

In [218]:
prueba=rvadd.copy()
ac=[]
for i in prueba["sic"]:
    if esta_en(i,indiceinter):
        ac.append(True)
    else:
        ac.append(False)
prueba["esta"]=ac
prueba=prueba[prueba["esta"]==True]
prueba=prueba.drop("esta",axis=1)
rvadd_inter=prueba.copy()
rvadd_inter=rvadd_inter.drop("vadd",axis=1)
rvadd_inter=rvadd_inter.drop("piship",axis=1)
ac=[]
for i in rvadd_inter["rvadd"]:
    ac.append(mt.log(i))
rvadd_inter["ln_rvadd"]=ac

In [219]:
rvadd_inter.head()

,sic,year,rvadd,ln_rvadd
33,2011,1991.0,5203.144191,8.557018
34,2011,1992.0,6379.374051,8.760825
35,2011,1993.0,6749.955541,8.817291
36,2011,1994.0,7825.679244,8.965166
37,2011,1995.0,8493.231448,9.047025


At the moment, we have $\ln Y_{i,t}$ and Input and Output. We do now need $Shock_{i,t}$ and later _downstream_ and _upstream_. In this case, $Shock_{i,t}$ is the China Trade as:  $ChinaTrade^{IV}_{j,t}=-\dfrac{NonUS\;Imports\;From\;China_{j,t}}{US\;Market\;Size_{j,1991}}$ due to exogenety _(data in data3)_.

In [220]:
data3.columns

Index(['sic87dd', 'year', 'l_import_otch'], dtype='object')

In [221]:
data3.head() #here we find directly the variable in diff 

,sic87dd,year,l_import_otch
0,2011.0,1991,0.195554
1,2011.0,1992,0.253203
2,2011.0,1993,0.230878
3,2011.0,1994,0.241313
4,2011.0,1995,0.308338


In [222]:
chinatrade=data3.loc[:,['sic87dd','year','d_import_otch_1991_2009']] #### No cambia entre años!!! 
## Ojo que igual es con un menos delante!!! (ya con menos)

In [223]:
chinatrade=data3.drop("l_import_otch",axis=1)

In [224]:
chinatrade["d_import_otch_1991_2009"]=data3["l_import_otch"]

In [225]:
chinatradeaux=[0]
i=1
while i<len(chinatrade):
    v=chinatrade["d_import_otch_1991_2009"][i]-chinatrade["d_import_otch_1991_2009"][i-1]
    chinatradeaux.append(v)
    i=i+1
chinatrade["d_import_otch_1991_2009"]=chinatradeaux
for i in chinatrade.index:
    if chinatrade["year"][i]==1991:
        chinatrade["d_import_otch_1991_2009"][i]="NaN"

/Users/ivanrendobarreiro/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [226]:
chinatrade

,sic87dd,year,d_import_otch_1991_2009
0,2011.0,1991,NaN
1,2011.0,1992,0.057648
2,2011.0,1993,-0.022324
3,2011.0,1994,0.010434
4,2011.0,1995,0.067025
5,2011.0,1996,0.020542
6,2011.0,1997,0.010874
7,2011.0,1998,0.013100
8,2011.0,1999,-0.015943
9,2011.0,2000,0.021565


Thus, now we have $Shock_{i,t}=\nabla ChinaTrade^{IV}_{i,t}$ (in logs)

### Downstream and Upstream

I need to create a matrix for $\nabla ChinaTrade$ where in rows we have years and in columns industries.

In [227]:
chinatrade=chinatrade[chinatrade["year"]<2010]

In [228]:
len(chinatrade.groupby("sic87dd").count().index)

392

In [229]:
len(indiceinter) ## COINCIDE !!! PORQUE ACEMOGLU HABRÁ HECHO LO MISMO

392

In [230]:
years=range(1991,2010)

In [231]:
matshock=pd.DataFrame()
matshock["year"]=years
matshock=matshock.set_index("year")

In [232]:
for i in indiceinter:
    serie=chinatrade[chinatrade["sic87dd"]==i].set_index("year")["d_import_otch_1991_2009"]
    matshock[i]=serie

In [233]:
matshock #this is matrix of shocks

,2011,2015,2021,2022,2023,2024,2026,2032,2033,2034,...,3951,3952,3953,3955,3961,3965,3991,3993,3996,3999
year,,,,,,,,,,,,,,,,,,,,,
1991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1992,0.057648,0.245912,0.000082,-4.494138e-05,0.017965,0.000000,0.000262,0.002230,0.247456,1.267667,...,0.371219,0.260164,0.050629,0.967950,0.609855,0.095097,0.999728,0.005902,-0.006106,0.978727
1993,-0.022324,0.254411,-0.000082,2.144024e-04,0.014568,0.000000,0.000003,0.007578,-0.067626,0.971472,...,-0.142918,0.578559,0.207325,0.551251,0.214918,0.140283,0.439156,0.000130,0.069564,0.077826
1994,0.010434,0.816252,0.000000,1.880172e-03,0.140970,0.000000,0.003914,0.012922,0.197867,1.112112,...,0.504753,0.168382,0.049260,1.162644,0.374650,0.550106,0.873781,0.015037,-0.001030,1.275111
1995,0.067025,1.029656,0.000000,-2.007401e-03,-0.024563,0.000000,-0.001142,0.006540,0.770479,-0.270903,...,0.858411,0.582299,0.070590,0.502137,0.266967,0.531937,1.031195,0.023972,-0.008837,0.642041
1996,0.020542,0.186937,0.000000,-9.181083e-05,0.027988,0.000000,0.001281,0.009009,-0.404595,0.518126,...,-0.094722,0.116571,0.603157,0.140614,-0.006078,0.450329,0.506910,0.005810,-0.032903,0.052062
1997,0.010874,-0.430709,0.000000,3.513233e-07,0.032465,0.000001,0.000937,-0.000479,-0.353862,1.142362,...,0.229678,-0.086975,-0.332902,0.280255,-0.083069,0.070781,0.445433,0.029976,0.030259,0.111364
1998,0.013100,-0.138735,0.000000,1.676299e-05,0.046493,-0.000001,-0.000014,-0.006899,-0.070612,-1.110846,...,0.273962,0.167561,0.593495,-0.956603,0.342234,0.090040,0.464097,-0.025350,0.061207,-0.418492
1999,-0.015943,0.254368,0.000000,2.486399e-05,0.011162,0.006399,0.001955,0.010911,0.382330,0.793346,...,0.697823,0.356955,-0.127296,-0.458937,0.564405,-0.080599,0.580596,0.050010,0.030350,0.667461


#### Pure Downstream and Upstream

As we know, $Downstream_{i,t}^{TradeIV}=\sum_j(Input\%^{1991}_{j\rightarrow i}-1_{j=i})·\nabla ChinaTrade_{j,t}^{IV}$. 
 
 First lets define Amid = A_in - Id, i.e.: $(Input\%^{1991}_{j\rightarrow i}-1_{j=i})$

In [234]:
identidad=pd.DataFrame()
for i in Input.index:
    identidad[i]=np.zeros(len(Input.index))
identidad["indice"]=Input.index
identidad=identidad.set_index("indice")
i=0
while i<len(identidad):
    identidad.iloc[i,i]=1
    i=i+1

In [235]:
Amid=Input-identidad

In [236]:
def valorprop(industria,anyo):
    valinput=Amid.loc[industria]
    valshock=matshock.loc[anyo]
    valor=sum(valinput*valshock)
    return valor

In [237]:
downstream=matshock.copy()
for i in downstream:
    downstream[i]=np.zeros(len(downstream))
for year in downstream.index:
    for ind in downstream.columns:
        downstream[ind][year]=valorprop(ind,year)
    

In [238]:
downstream #la mayoría son negativos, quizás por el signo???

,2011,2015,2021,2022,2023,2024,2026,2032,2033,2034,...,3951,3952,3953,3955,3961,3965,3991,3993,3996,3999
year,,,,,,,,,,,,,,,,,,,,,
1991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1992,0.167371,0.100439,0.008552,0.250651,0.154124,0.057491,0.081350,0.111496,0.120885,0.211406,...,0.067357,0.026325,0.004797,0.010900,0.004083,-0.018076,0.010607,0.035329,0.008562,-0.027183
1993,0.242894,0.151209,0.010429,0.039095,-0.012113,0.053222,0.116833,0.139370,0.220449,0.181852,...,0.014228,0.023677,0.009234,0.032013,0.023310,0.037481,0.030353,0.110632,0.017027,0.309881
1994,0.661986,0.394641,0.028188,0.231765,0.093122,0.154325,0.302448,0.665915,0.761537,1.236828,...,0.123643,0.083142,0.023445,0.042259,0.056826,0.075988,0.076857,0.255599,0.051697,0.530853
1995,1.410537,0.805855,0.068979,0.611975,0.252856,0.368224,0.773647,0.551694,1.688686,-0.771691,...,0.239637,0.150221,0.054013,0.088381,0.120743,0.199940,0.158067,0.641523,0.118631,1.530253
1996,-0.049483,-0.001225,-0.003667,0.043668,0.010578,-0.012667,-0.054648,-0.170157,-0.409773,0.087594,...,-0.021913,-0.014560,-0.001490,0.009567,-0.017850,-0.063580,-0.014343,-0.070237,-0.009133,-0.422024
1997,0.317473,0.085299,0.016703,0.216115,0.124841,0.086915,0.183040,0.335494,0.574242,0.236225,...,0.050356,0.017929,0.006917,0.023362,0.018622,0.008483,0.021305,0.135072,0.027614,-0.078120
1998,-0.334038,-0.160217,-0.015631,-0.318472,-0.194293,-0.096816,-0.150197,-0.260335,-0.444776,-0.178906,...,0.015781,-0.024481,-0.013006,-0.048555,-0.022379,-0.040331,-0.045510,-0.145464,-0.027428,-0.408524
1999,0.254154,0.156065,0.009466,0.110370,0.026260,0.054759,0.087586,0.086497,0.144836,0.141922,...,0.126561,0.041898,0.006477,0.020685,0.025349,-0.006224,0.024110,0.095305,0.013648,0.132884


Now, $Upstream_{i,t}=\sum_j(Output\%_{i\rightarrow j}^{1991}-1_{j=i})·Shock_{j,t}$. 

In [239]:
def valorprop2(industria,anyo):
    valinput=Omid.loc[industria]
    valshock=matshock.loc[anyo]
    valor=sum(valinput*valshock)
    return valor

In [240]:
Omid=Output-identidad
upstream=matshock.copy()
for i in upstream:
    upstream[i]=np.zeros(len(downstream))
for year in upstream.index:
    for ind in upstream.columns:
        upstream[ind][year]=valorprop2(ind,year)

In [241]:
upstream #casi todos negativos otra vez, casi seguro de que hay que cambiar el signo arriba en la definición de trade ¿igual que cambiarlo abajo?
## Ojo, no debe de estar mal, porque está restado el shock propio ¡¡positivo!!

,2011,2015,2021,2022,2023,2024,2026,2032,2033,2034,...,3951,3952,3953,3955,3961,3965,3991,3993,3996,3999
year,,,,,,,,,,,,,,,,,,,,,
1991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1992,2.380744,0.266370,0.006370,0.168980,0.058508,0.043492,0.119452,0.016347,0.081287,0.198121,...,0.089592,0.047758,0.021519,0.025124,0.007514,0.567481,0.030487,0.152327,0.039252,0.222468
1993,2.357895,0.239107,0.006169,0.160947,0.054881,0.041447,0.108613,0.016610,0.064104,0.155146,...,0.025223,0.037461,0.020542,0.022151,0.003742,0.377627,0.026376,0.139745,0.037632,0.175260
1994,4.412631,0.586504,0.014236,0.375251,0.138338,0.096301,0.245360,0.038977,0.164631,0.191785,...,0.134554,0.085741,0.046532,0.056148,0.006994,0.674171,0.062539,0.324742,0.088778,0.425089
1995,4.474971,0.620015,0.013213,0.351852,0.116163,0.091587,0.193526,0.039156,0.179928,-0.005527,...,0.198204,0.093214,0.041765,0.046504,0.006247,0.736678,0.058193,0.298181,0.081310,0.383528
1996,1.283542,0.271201,0.007914,0.211489,0.072335,0.053408,0.132005,0.018422,0.069391,0.091401,...,0.021936,0.044945,0.029366,0.030825,0.003308,0.740328,0.036598,0.193743,0.047580,0.226949
1997,-1.531657,-0.099605,-0.000818,-0.028285,-0.005658,-0.006539,0.005717,-0.001671,-0.025824,0.160486,...,0.025202,-0.007159,-0.006394,-0.003744,-0.002596,-0.681255,-0.000233,-0.012380,-0.006327,-0.038486
1998,-3.473190,-0.277461,-0.006491,-0.168672,-0.052376,-0.043253,-0.110367,-0.022773,-0.072722,-0.176811,...,0.022947,-0.025794,-0.017487,-0.025166,0.000508,-0.520129,-0.023592,-0.137378,-0.040267,-0.164356
1999,1.607205,0.355120,0.009993,0.267485,0.091060,0.068995,0.169214,0.024952,0.128994,0.138825,...,0.155540,0.075329,0.032563,0.039152,0.008540,0.800703,0.044760,0.233322,0.062638,0.314200


#### Preparing $\nabla \ln Y_{i,t}$

### Estimation

Finally, we will estimate trought fixed effects the follwing equation: 
 
 $\Delta\ln Y_{i,t}=\delta_t+\psi\Delta\ln Y_{i,t-1}+\beta^{own}Shock_{i,t-1}+\beta^{upstream}Upstream_{i,t-1}+\beta^{downstream}Downstream_{i,t-1}+\varepsilon_{i,t}\;\;\forall_i$

In [243]:
yt=pd.DataFrame()
i=0
while i<=(len(rvadd_inter)-1):
    industria=rvadd_inter.iloc[i:i+19]["ln_rvadd"].values
    ind=rvadd_inter.iloc[i]["sic"]
    yt[ind]=industria
    i=i+19
yt["year"]=range(1991,2010)
yt=yt.set_index("year")

In [244]:
lnyt=yt.copy()

In [245]:
dlnyt=yt.copy()

In [246]:
for y in lnyt.index:
    for ind in lnyt.columns:
        if y==1991:
            dlnyt[ind][y]=0
        else:
            dlnyt[ind][y]=lnyt[ind][y]-lnyt[ind][y-1]

In [247]:
dlnyt=dlnyt.drop(1991,axis=0)

In [248]:
dlnyt.head()

,2011.0,2015.0,2021.0,2022.0,2023.0,2024.0,2026.0,2032.0,2033.0,2034.0,...,3951.0,3952.0,3953.0,3955.0,3961.0,3965.0,3991.0,3993.0,3996.0,3999.0
year,,,,,,,,,,,,,,,,,,,,,
1992,0.203807,0.042149,0.155304,0.180393,0.104688,0.273253,0.019695,0.021987,0.007386,0.115444,...,-0.015923,0.229657,0.129334,0.000154,0.042452,0.047028,-0.013059,0.199266,0.104543,0.051158
1993,0.056466,0.113540,0.276822,-0.153235,0.004207,0.103027,0.021668,0.019485,-0.006141,-0.209870,...,-0.025393,0.034703,-0.090715,-0.106612,0.015042,0.062321,0.055850,0.014894,0.008710,0.054019
1994,0.147875,0.055617,-0.101100,-0.095771,-0.097332,0.018771,-0.017301,0.089576,-0.074789,-0.019312,...,0.025734,0.037953,0.377677,0.282927,0.102234,-0.064750,-0.003947,0.072217,0.062770,0.026725
1995,0.081859,0.220966,0.057830,0.047426,-0.089733,0.011156,0.046404,0.010892,-0.100038,0.006722,...,0.023341,0.034022,0.202930,-0.131981,0.031689,-0.139768,-0.124028,0.061632,-0.096714,0.021756
1996,-0.118199,-0.164160,0.026371,-0.111150,-0.063489,0.052182,-0.095337,0.033111,0.013052,-0.088087,...,0.105649,0.007465,-0.046518,0.144445,-0.097947,0.000875,0.045923,0.041359,0.077521,0.084072


In [249]:
dlnyt

,2011.0,2015.0,2021.0,2022.0,2023.0,2024.0,2026.0,2032.0,2033.0,2034.0,...,3951.0,3952.0,3953.0,3955.0,3961.0,3965.0,3991.0,3993.0,3996.0,3999.0
year,,,,,,,,,,,,,,,,,,,,,
1992,0.203807,0.042149,0.155304,0.180393,0.104688,0.273253,0.019695,0.021987,0.007386,0.115444,...,-0.015923,0.229657,0.129334,0.000154,0.042452,0.047028,-0.013059,0.199266,0.104543,0.051158
1993,0.056466,0.113540,0.276822,-0.153235,0.004207,0.103027,0.021668,0.019485,-0.006141,-0.209870,...,-0.025393,0.034703,-0.090715,-0.106612,0.015042,0.062321,0.055850,0.014894,0.008710,0.054019
1994,0.147875,0.055617,-0.101100,-0.095771,-0.097332,0.018771,-0.017301,0.089576,-0.074789,-0.019312,...,0.025734,0.037953,0.377677,0.282927,0.102234,-0.064750,-0.003947,0.072217,0.062770,0.026725
1995,0.081859,0.220966,0.057830,0.047426,-0.089733,0.011156,0.046404,0.010892,-0.100038,0.006722,...,0.023341,0.034022,0.202930,-0.131981,0.031689,-0.139768,-0.124028,0.061632,-0.096714,0.021756
1996,-0.118199,-0.164160,0.026371,-0.111150,-0.063489,0.052182,-0.095337,0.033111,0.013052,-0.088087,...,0.105649,0.007465,-0.046518,0.144445,-0.097947,0.000875,0.045923,0.041359,0.077521,0.084072
1997,0.012297,0.343052,0.007982,0.234419,0.356004,-0.088297,0.008395,0.088102,0.002963,0.083985,...,0.039968,-0.137625,-0.359701,-0.093303,-0.230016,0.099271,0.160604,0.074816,-0.119023,-0.001907
1998,0.053808,0.119773,-0.249127,0.124639,-0.037212,-0.177624,0.016373,0.020677,0.084505,-0.057504,...,0.130701,-0.083401,0.005583,-0.129162,-0.006903,-0.070844,0.110349,0.000534,-0.031934,0.042605
1999,0.143419,0.066336,0.127744,-0.083536,-0.042283,-0.046532,0.044691,-0.016527,0.033119,0.226922,...,-0.124941,0.008667,0.108081,-0.145230,-0.192036,-0.017547,-0.030181,0.104120,-0.028000,0.044838
2000,-0.037124,-0.006521,-0.009406,0.050255,0.040239,0.302211,0.107096,-0.042422,0.026441,-0.101391,...,-0.024057,-0.180787,-0.071699,0.019210,0.157347,-0.029074,-0.052601,-0.005525,-0.166883,-0.000218


In [250]:
data=pd.DataFrame()

In [251]:
data["dlnyt"]=[]
data["industry"]=[]
data["year"]=[]

In [252]:
for i in dlnyt.columns:
    for j in dlnyt.index:
        parcial=pd.DataFrame()
        parcial["dlnyt"]=[dlnyt[i][j]]
        parcial["industry"]=[i]
        parcial["year"]=[j]
        data=pd.concat([data,parcial])
data["indice"]=range(0,len(data))
data=data.set_index("indice")
ac=[]
for i in data.index:
    if data["year"][i]==1992:
        ac.append(0)
    else:
        ac.append(data["dlnyt"][i-1])
data["dlnyt_1"]=ac

In [253]:
data

,dlnyt,industry,year,dlnyt_1
indice,,,,
0,0.203807,2011.0,1992.0,0.000000
1,0.056466,2011.0,1993.0,0.203807
2,0.147875,2011.0,1994.0,0.056466
3,0.081859,2011.0,1995.0,0.147875
4,-0.118199,2011.0,1996.0,0.081859
5,0.012297,2011.0,1997.0,-0.118199
6,0.053808,2011.0,1998.0,0.012297
7,0.143419,2011.0,1999.0,0.053808
8,-0.037124,2011.0,2000.0,0.143419


In [254]:
data2=pd.DataFrame()
data2["upstream"]=[]
data2["industry"]=[]
data2["year"]=[]
for i in upstream.columns:
    for j in upstream.index:
        parcial=pd.DataFrame()
        parcial["upstream"]=[upstream[i][j]]
        parcial["industry"]=[i]
        parcial["year"]=[j]
        data2=pd.concat([data2,parcial])
data2["indice"]=range(0,len(data2))
data2=data2.set_index("indice")

In [255]:
data3=pd.DataFrame()
data3["downstream"]=[]
data3["industry"]=[]
data3["year"]=[]
for i in downstream.columns:
    for j in downstream.index:
        parcial=pd.DataFrame()
        parcial["downstream"]=[downstream[i][j]]
        parcial["industry"]=[i]
        parcial["year"]=[j]
        data3=pd.concat([data3,parcial])
data3["indice"]=range(0,len(data3))
data3=data3.set_index("indice")

In [256]:
data4=pd.DataFrame()
data4["ownshock"]=[]
data4["industry"]=[]
data4["year"]=[]
for i in downstream.columns:
    for j in downstream.index:
        parcial=pd.DataFrame()
        parcial["ownshock"]=[matshock[i][j]]
        parcial["industry"]=[i]
        parcial["year"]=[j]
        data4=pd.concat([data4,parcial])
data4["indice"]=range(0,len(data4))
data4=data4.set_index("indice")

In [257]:
data2=data2[data2["year"]!=1991]
data2["indice"]=range(0,len(data2))
data2=data2.set_index("indice")
data3=data3[data3["year"]!=1991]
data3["indice"]=range(0,len(data3))
data3=data3.set_index("indice")
data4=data4[data4["year"]!=1991]
data4["indice"]=range(0,len(data4))
data4=data4.set_index("indice")
data2=data2.drop(["year","industry"],axis=1)
data3=data3.drop(["year","industry"],axis=1)
data4=data4.drop(["year","industry"],axis=1)


In [258]:
data=pd.concat([data,data2,data3,data4],axis=1)

In [259]:
data #finally here is it, now the estimation!

,dlnyt,industry,year,dlnyt_1,upstream,downstream,ownshock
indice,,,,,,,
0,0.203807,2011.0,1992.0,0.000000,2.380744,0.167371,0.057648
1,0.056466,2011.0,1993.0,0.203807,2.357895,0.242894,-0.022324
2,0.147875,2011.0,1994.0,0.056466,4.412631,0.661986,0.010434
3,0.081859,2011.0,1995.0,0.147875,4.474971,1.410537,0.067025
4,-0.118199,2011.0,1996.0,0.081859,1.283542,-0.049483,0.020542
5,0.012297,2011.0,1997.0,-0.118199,-1.531657,0.317473,0.010874
6,0.053808,2011.0,1998.0,0.012297,-3.473190,-0.334038,0.013100
7,0.143419,2011.0,1999.0,0.053808,1.607205,0.254154,-0.015943
8,-0.037124,2011.0,2000.0,0.143419,3.002494,0.952340,0.021565


In [260]:
### need to move upstream(t) to upstream(t-1) etc

In [261]:
ac1=[0]
ac2=[0]
ac3=[0]

for i in data.index[1:]:
    ac1.append(data["upstream"][i-1])
    ac2.append(data["downstream"][i-1])
    ac3.append(data["ownshock"][i-1])
data["upstream"]=ac1
data["downstream"]=ac2
data["ownshock"]=ac3

In [262]:
data.head()

,dlnyt,industry,year,dlnyt_1,upstream,downstream,ownshock
indice,,,,,,,
0,0.203807,2011.0,1992.0,0.000000,0.000000,0.000000,0.000000
1,0.056466,2011.0,1993.0,0.203807,2.380744,0.167371,0.057648
2,0.147875,2011.0,1994.0,0.056466,2.357895,0.242894,-0.022324
3,0.081859,2011.0,1995.0,0.147875,4.412631,0.661986,0.010434
4,-0.118199,2011.0,1996.0,0.081859,4.474971,1.410537,0.067025


In [263]:
data=data[data["year"]!=1992]

In [264]:
data=data.set_index(["industry","year"])

In [265]:
len(data)

6664

**ESTIMATION** via Fixed Effects

In [266]:
mod = PanelOLS(data.dlnyt, data[['dlnyt_1','upstream','downstream','ownshock']])
res = mod.fit(cov_type='clustered', cluster_entity=True)

In [267]:
mod

PanelOLS 
Num exog: 4, Constant: False
Entity Effects: False, Time Effects: False, Num Other Effects: 0
id: 0x1a18aab630

In [268]:
res

Dep. Variable:,dlnyt,R-squared:,0.0207
Estimator:,PanelOLS,R-squared (Between):,0.1555
No. Observations:,6573,R-squared (Within):,0.0086
Date:,"Tue, Dec 31 2019",R-squared (Overall):,0.0207
Time:,20:25:05,Log-likelihood,2797.5
Cov. Estimator:,Clustered,,
,,F-statistic:,34.630
Entities:,392,P-value,0.0000
Avg Obs:,16.768,Distribution:,"F(4,6569)"
Min Obs:,4.0000,,
Max Obs:,17.000,F-statistic (robust):,10.891
